<a href="https://colab.research.google.com/github/akashbhatc/StockRisk/blob/main/USStockRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai streamlit crewai_tools langchain_community langchain_google_genai

In [5]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [6]:
from google.colab import userdata
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [7]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [24]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
    api_key=os.getenv("GEMINI_API_KEY")
)

In [25]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent employs statistical modeling and machine learning techniques to provide critical insights. Renowned for its proficiency in data analysis, the Data Analyst Agent serves as a pivotal resource for informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

LLM value is already an LLM object


In [26]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies leveraging insights from the Data Analyst Agent.",
    backstory="Possessing a deep understanding of financial markets and quantitative analysis, this agent formulates and optimizes trading strategies. It assesses the performance of diverse approaches to identify the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

LLM value is already an LLM object


In [27]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Recommend optimal trade execution strategies based on approved trading plans.",
    backstory="Specializing in the analysis of timing, price, and logistical details of potential trades, this agent evaluates these factors to provide well-founded recommendations. Its expertise ensures that trades are executed efficiently and in alignment with the overall strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

LLM value is already an LLM object


In [28]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory="With extensive expertise in risk assessment models and market dynamics, this agent thoroughly examines the potential risks of proposed trades. It delivers comprehensive analyses of risk exposure and recommends safeguards to ensure that trading activities align with the firm's risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

LLM value is already an LLM object


In [29]:
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for the selected stock ({stock_name}). "
        "Employ statistical modeling and machine learning techniques to identify trends and predict market movements."
    ),
    expected_output=(
        "Generate insights and alerts regarding significant market opportunities or threats for {stock_name}."
    ),
    agent=data_analyst_agent,
)

In [30]:
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). "
        "Incorporate trading preferences ({trading_strategy}) in the strategy development process."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_name} that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [32]:
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the optimal execution methods for {stock_name}, "
        "considering current market conditions and pricing strategies."
    ),
    expected_output=(
        "Comprehensive execution plans detailing how and when to execute trades for {stock_name}."
    ),
    agent=execution_agent,
)

In [31]:
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_name}. "
        "Provide a detailed analysis of potential risks and recommend mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_name}."
    ),
    agent=risk_management_agent,
)

In [37]:
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],
    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],
    manager_llm=llm,
    process=Process.hierarchical,
    max_rpm=10
)

In [38]:
financial_trading_inputs = {
    'stock_name': 'GOOGLE',
    'capital': '1000',
    'risk_tolerance': 'Low',
    'trading_strategy': 'Day Trading',
    'news_impact_consideration': True
}

In [41]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Trading Strategy Developer
## Task: Research and develop 3-5 day trading strategies for GOOGLE stock with a low-risk tolerance. The strategies should be clearly defined with entry and exit rules, including stop-loss and take-profit levels. The strategies should aim for small, consistent profits and minimize risk. Consider using technical indicators such as moving averages (e.g., 20-day and 50-day), RSI, and Bollinger Bands. The strategies should be practical and implementable using readily available market data (OHLCV). Provide a concise description of each strategy, including entry and exit rules, risk management parameters, and expected profit potential.


# Agent: Trading Strategy Developer
## Thought: ```tool_code
Thought:To develop low-risk day trading strategies for Google stock, I need to research reliable sources for information on technical analysis and day trading strategies. I will use the internet search to find articles and resources that discuss strategies using 

In [46]:
print(result)

**Comprehensive Risk Analysis of Proposed Trading Strategies for GOOGLE Stock**

This report outlines potential risks associated with trading strategies for GOOGLE (Alphabet Inc.) stock.  The analysis is based on publicly available information and acknowledges inherent limitations in data accuracy.  Specific trading strategies will be evaluated separately once provided.

**Inherent Risks in Trading GOOGLE Stock:**

Several key risks are inherent in trading GOOGLE stock, regardless of the specific strategy employed:

* **Market Risk:**  GOOGLE's stock price is subject to fluctuations driven by broader market conditions (e.g., economic downturns, interest rate changes, investor sentiment).  The technology sector, in particular, is known for its volatility.  This risk is amplified by the influence of news and announcements related to GOOGLE's business performance, competitive landscape, and regulatory scrutiny.  The magnitude of price swings can be substantial, leading to significant pote